Stage 2:  Data preprocessing stage to produce structured data in csv format also stored in Cloud Object Storage. As columns in the csv file we suggest date, geographic location, url, and sentiment analysis.

In [1]:
import lithops
from transformers import pipeline

In [2]:
# https://github.com/lithops-cloud/lithops/blob/master/docs/data_processing.md

In [6]:
def map_preprocess(obj):

    # TODO: Investigate why this is not generating any logs...

    print('Bucket: {}'.format(obj.bucket))
    print('Key: {}'.format(obj.key))
    print('Partition num: {}'.format(obj.part))

    data_body_str = obj.data_stream.read().decode('utf-8')

    # TODO: Serialize tweet dataclass from str dict in data_body HERE


    # this model is the best one, I tested the others available and they're quite bad...
    #classifier = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")
    #results = classifier(tweet.content)
    return data_body_str #results

In [7]:
def reduce_to_csv(results):
    # TODO: Save results to a single csv with columns as the first row

    print(results)

    return "reduced result to csv output and stored it"

In [8]:
data_location = ['colab/#coronavirus/', 'colab/#sars/', 'colab/#covid/'] # The / at the end is important TODO: Avoid repetition?

with lithops.FunctionExecutor() as fexec: #runtime='gilarasa/lithops-cloudbutton-challenge-py3.9:0.1', runtime_memory=2048) as fexec:

  fexec.map_reduce(map_preprocess, data_location, reduce_to_csv, obj_chunk_number=4) # 4 inferences per function
  print(fexec.get_result())

2021-05-19 22:31:48,923 [INFO] lithops.config -- Lithops v2.3.3
2021-05-19 22:31:48,936 [INFO] lithops.storage.backends.ibm_cos.ibm_cos -- IBM COS Storage client created - Region: eu-de
2021-05-19 22:31:48,939 [INFO] lithops.serverless.backends.ibm_cf.ibm_cf -- IBM CF client created - Region: eu-de - Namespace: Namespace-n4e-colab
2021-05-19 22:31:48,940 [INFO] lithops.executors -- Serverless Executor created with ID: 2fbdc8-1
2021-05-19 22:31:48,940 [INFO] lithops.invokers -- ExecutorID 2fbdc8-1 | JobID M000 - Selected Runtime: lithopscloud/ibmcf-python-v39 - 256MB
2021-05-19 22:31:50,229 [INFO] lithops.job.job -- ExecutorID 2fbdc8-1 | JobID M000 - Uploading function and data - Total: 5.3KiB
2021-05-19 22:31:50,640 [INFO] lithops.invokers -- ExecutorID 2fbdc8-1 | JobID M000 - Starting function invocation: map_preprocess() - Total: 24 activations
2021-05-19 22:31:50,682 [INFO] lithops.invokers -- ExecutorID 2fbdc8-1 | JobID M000 - View execution logs at /tmp/lithops/logs/2fbdc8-1-M000.

In [ ]:
!lithops logs poll
